In [ ]:
import numpy as np
import tensorflow as tf
from model import Model
import os
from pykalman import KalmanFilter

In [ ]:
with open('data.txt') as f:
    first = f.readline()
    first = first.strip('\n')
    temp = first.split(' ')
    T = int(temp[0])
    o_dim = int(temp[1])
    s_dim = int(temp[2])
    o_matrix = np.zeros((T, o_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(s_dim):
            o_matrix[i,j] = float(temp[j])
    s_matrix = np.zeros((T, s_dim), np.float32)
    for i in range(T):
        temp = f.readline().strip('\n').split(' ')
        for j in range(o_dim):
            s_matrix[i,j] = float(temp[j])

In [ ]:
h_dim = 10
train_data = np.zeros((int(T/2), 3*s_dim), np.float32)
for i in range(int(T/2)):
    train_data[i, :] = np.concatenate((s_matrix[i, :], s_matrix[i+1, :], o_matrix[i+1, :]), axis=0)
test_data = np.zeros((int(T/2) - 1, 3*s_dim), np.float32)
for i in range(int(T/2) - 1):
    test_data[i, :] = np.concatenate((s_matrix[int(T/2) + i, :], s_matrix[int(T/2) + 1 + i, :], o_matrix[int(T/2) + 1 + i, :]),0)

In [ ]:
class Dataset(object):
    def __init__(self, train, test):
        self.train = train
        self.test = test
    def random_batch(self, batch_size):
        index = np.random.choice(np.arange(len(self.train)),batch_size, False)
        return self.train[index,:]
dataset = Dataset(train_data, test_data)

In [ ]:
log_dir = './log/'
os.popen('rm '+log_dir+'*')
minibatch_size = 256
model = Model(s_dim, h_dim, minibatch_size, 1e-4, log_dir)
iteration = 300
for epoch in range(85):
    reconstruction_loss_train, likelihood_train, classify_loss_train = 0., 0., 0.
    global_step = tf.contrib.framework.get_or_create_global_step()
    # in each epoch 500 iterations
    for i in range(iteration):
        reconstruction_loss_value, likelihood_value, classify_loss_value, summary = \
                model.update_params(dataset.random_batch(minibatch_size))
            
        reconstruction_loss_train += reconstruction_loss_value
        likelihood_train += likelihood_value
        classify_loss_train += classify_loss_value
        model.train_writer.add_summary(summary, global_step.eval(model.sess))
    
    reconstruction_loss_train = reconstruction_loss_train / (iteration)
    likelihood_train = -likelihood_train / (iteration)
    classify_loss_train = classify_loss_train / (iteration)
    

    print("step: {},\trecons loss: {:.4f},\tlikelihood: {:.4f},\tclass loss: {:.4f}".format(global_step.eval(model.sess),
            reconstruction_loss_train, likelihood_train, classify_loss_train))


In [ ]:
last_s_p = np.asarray(model.sess.run([model.s_t_p], {model.input_tensor: train_data[train_data.shape[0]-2 : ,:]}))[0][1,:]

a_2, sig_2, a_3, sig_3 = model.sess.run([model.a_2, model.sigma_2, model.a_3, model.sigma_3])
kf = KalmanFilter(initial_state_mean = np.transpose(np.matmul(last_s_p,a_2)), \
                  initial_state_covariance = sig_2, \
                  transition_matrices = np.transpose(a_2), \
                  transition_covariance = sig_2, \
                  observation_matrices = np.transpose(a_3),\
                  observation_covariance = sig_3)
est_o_t_p = model.sess.run(model.o_t_p, {model.input_tensor: test_data[:1000,]})
measurements = np.asarray(est_o_t_p)
(est_s_t_p, est_s_t_p_covariances) = kf.filter(measurements)
est_s_t = model.decode_s_t_p(est_s_t_p)
print(np.mean(np.linalg.norm(est_s_t - test_data[:1000, s_dim : 2*s_dim] , axis = 1)), end = '')
print(' / ', end = '')
print(np.mean(np.linalg.norm(test_data[:1000, s_dim : 2*s_dim], axis = 1)))
print('mean consecutive diff: ', end='')
print(np.mean(np.linalg.norm(test_data[1:1000, s_dim : 2*s_dim] - test_data[:999, s_dim : 2*s_dim], axis = 1)))